In [1]:
import pandas as pd
from CAD.config import Config

In [6]:
config = Config()
min_year, max_year = config.getMinMaxYear()

attendance_dfs = {}                        
for year in range(min_year, max_year + 1):
    fp = config.RAW_DATA_DIR / f"{year}_attendance_data.xlsx"
    if year != 2024:
        attendance_dfs[year] = pd.read_excel(fp, engine="openpyxl")
    else:
        attendance_dfs[year] = pd.read_excel(fp, engine="openpyxl", sheet_name='Sheet1')

In [7]:
attendance_dfs[2024].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299833 entries, 0 to 299832
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   SCHOOL_YEAR                  299833 non-null  datetime64[ns]
 1   DISTRICT_CODE                299833 non-null  int64         
 2   DISTRICT_NAME                299833 non-null  object        
 3   LOCATION_ID                  299833 non-null  int64         
 4   SCHOOL_NAME                  299833 non-null  object        
 5   STUDENT_ID                   299833 non-null  int64         
 6   TOTAL_DAYS_UNEXCUSED_ABSENT  299833 non-null  float64       
 7   TOTAL_DAYS_ENROLLED          299833 non-null  int64         
 8   TOTAL_DAYS_PRESENT           299833 non-null  float64       
 9   STUDENT_GRADE_LEVEL          299833 non-null  object        
 10  CURR_GRADE_ORD               299833 non-null  object        
 11  ETHNIC_CODE               

In [8]:
name_map = {
    'TOTAL_DAYS_UNEXCUSED_ABSENT': 'Total_Days_Unexcused_Absent',
    'TOTAL_DAYS_ENROLLED': 'Total_Days_Enrolled',
    'TOTAL_DAYS_PRESENT': 'Total_Days_Present'
}

attendance_dfs[2024] = attendance_dfs[2024].rename(name_map, axis=1)

In [9]:
mapper = {
    'A': 'Asian',
    'B': 'Black or African American',
    'C': 'Caucasian',
    'I': 'American Indian/Alaskan Native',
    'P': 'Native Hawaiian or Other Pacific Islander',
    'H': 'Hispanic',
    'MR': 'Multi-Racial'
}

attendance_dfs[2024]['ETHNIC_CODE'] = attendance_dfs[2024]['ETHNIC_CODE'].replace(list(mapper.values()), list(mapper.keys()))

In [10]:
attendance_dfs[2024]['ECONOMIC_CODE'] = attendance_dfs[2024]['ECONOMIC_CODE'].replace([0, 1], ['N', 'F'])
attendance_dfs[2024]['SPECIAL_ED_CODE'] = attendance_dfs[2024]['SPECIAL_ED_CODE'].replace([0, 1], ['N', 'Y'])
attendance_dfs[2024]['HISPANIC_IND'] = attendance_dfs[2024]['HISPANIC_IND'].replace([0, 1], ['No', 'Yes'])
attendance_dfs[2024]['STUDENT_GENDER'] = attendance_dfs[2024]['STUDENT_GENDER'].replace(['M', 'F'], ['Male', 'Female'])
attendance_dfs[2024]['STUDENT_GRADE_LEVEL'] = attendance_dfs[2024]['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1]).astype('int')

/var/folders/g7/qvkgm1k12kzfvx8hv5_jqcph0000gn/T/ipykernel_53319/2272413774.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  attendance_dfs[2024]['STUDENT_GRADE_LEVEL'] = attendance_dfs[2024]['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1]).astype('int')


In [11]:
duplicated_students =  attendance_dfs[2024][attendance_dfs[2024]['STUDENT_ID'].duplicated(keep=False)].copy()

grade_pattern_ids = []
school_pattern_ids = []
grade_and_school_pattern_ids = []
ethnic_pattern_ids = []

for student, group in duplicated_students.groupby('STUDENT_ID'):
    if len(group) < 2:
        continue

    if group['STUDENT_GRADE_LEVEL'].nunique() > 1 and group['SCHOOL_NAME'].nunique() == 1:
        grade_pattern_ids.append(student)

    elif group['SCHOOL_NAME'].nunique() > 1 and group['STUDENT_GRADE_LEVEL'].nunique() == 1:
        school_pattern_ids.append(student)
    
    elif group['SCHOOL_NAME'].nunique() > 1 and group['STUDENT_GRADE_LEVEL'].nunique() > 1:
        grade_and_school_pattern_ids.append(student)
    
    else:
        ethnic_pattern_ids.append(student)


grade_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(grade_pattern_ids)]
school_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(school_pattern_ids)]
grade_and_school_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(grade_and_school_pattern_ids)]
ethnic_pattern_df = duplicated_students[duplicated_students['STUDENT_ID'].isin(ethnic_pattern_ids)]

In [12]:
ethnic_pattern_df = ethnic_pattern_df.drop_duplicates(subset='STUDENT_ID', keep='first')
school_pattern_df = school_pattern_df.drop_duplicates(subset=['STUDENT_ID', 'SCHOOL_NAME'], keep='last').fillna('C')
grade_and_school_pattern_df = grade_and_school_pattern_df.drop_duplicates(subset='STUDENT_ID', keep='last')

In [13]:
attendance_dfs[2023]['STUDENT_GRADE_LEVEL'] = attendance_dfs[2023]['STUDENT_GRADE_LEVEL'].replace(['KF', 'PK'], [0, -1])
data_2023_unique = attendance_dfs[2023].drop_duplicates(subset=['STUDENT_ID'], keep='first')

df = (
    grade_pattern_df
    .merge(data_2023_unique, on="STUDENT_ID", how="left", suffixes=("", "_prev"))
)

df["expected_grade"] = df["STUDENT_GRADE_LEVEL_prev"] + 1
df["has_prev"] = df["STUDENT_GRADE_LEVEL_prev"].notna()

keepers = df.loc[
      (df["STUDENT_GRADE_LEVEL"] == df["expected_grade"])
   | (~df["has_prev"] & (df["STUDENT_GRADE_LEVEL"] == -1))
].copy()

flagged = df.drop(index=keepers.index).copy()

keepers = keepers[grade_pattern_df.columns]
flagged = flagged[grade_pattern_df.columns.to_list()]

In [14]:
flagged = flagged[~flagged['STUDENT_ID'].isin(keepers['STUDENT_ID'].values)]
idx = flagged.groupby('STUDENT_ID')['STUDENT_GRADE_LEVEL'].idxmax()
flagged = flagged.loc[idx].reset_index(drop=True)

In [15]:
dropping_ids = grade_and_school_pattern_ids + grade_pattern_ids + school_pattern_ids + ethnic_pattern_ids
attendance_dfs[2024] = attendance_dfs[2024][~attendance_dfs[2024]['STUDENT_ID'].isin(dropping_ids)]
attendance_dfs[2024] = pd.concat([attendance_dfs[2024], ethnic_pattern_df, school_pattern_df, keepers, flagged, grade_and_school_pattern_df], axis=0).reset_index(drop=True)

In [16]:
gender_lookup = (
    attendance_dfs[2023]
      .dropna(subset=["STUDENT_GENDER"])
      .groupby(["SCHOOL_NAME", "STUDENT_ID"], sort=False)["STUDENT_GENDER"]
      .first()
)

mask = attendance_dfs[2024]["STUDENT_GENDER"].isna()
keys = list(zip(attendance_dfs[2024].loc[mask, "SCHOOL_NAME"], attendance_dfs[2024].loc[mask, "STUDENT_ID"]))
attendance_dfs[2024].loc[mask, "STUDENT_GENDER"] = [gender_lookup.get(k, pd.NA) for k in keys]
attendance_dfs[2024]['STUDENT_GENDER'] = attendance_dfs[2024]['STUDENT_GENDER'].fillna(attendance_dfs[2024]['STUDENT_GENDER'].mode().values[0])
attendance_dfs[2024]['ETHNIC_CODE'] = attendance_dfs[2024]['ETHNIC_CODE'].fillna('C')
attendance_dfs[2024].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299511 entries, 0 to 299510
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   SCHOOL_YEAR                  299511 non-null  datetime64[ns]
 1   DISTRICT_CODE                299511 non-null  int64         
 2   DISTRICT_NAME                299511 non-null  object        
 3   LOCATION_ID                  299511 non-null  int64         
 4   SCHOOL_NAME                  299511 non-null  object        
 5   STUDENT_ID                   299511 non-null  int64         
 6   Total_Days_Unexcused_Absent  299511 non-null  float64       
 7   Total_Days_Enrolled          299511 non-null  int64         
 8   Total_Days_Present           299511 non-null  float64       
 9   STUDENT_GRADE_LEVEL          299511 non-null  int64         
 10  CURR_GRADE_ORD               299511 non-null  object        
 11  ETHNIC_CODE               

In [17]:
attendance_dfs[2024] = attendance_dfs[2024].drop(72159, axis=0)  #Dropped a record whose total days enrolled > 365

In [18]:
duplicated_past = attendance_dfs[2020][attendance_dfs[2020]['STUDENT_ID'].duplicated(keep=False)].sort_values(by='STUDENT_ID')

non_duplicates = []
others = []
duplicates = []
for student, group in duplicated_past.groupby('STUDENT_ID'):
    if len(group) < 2:
        continue

    if group['SCHOOL_NAME'].nunique() == 1:
        non_duplicates.append(student)
    
    elif group['SCHOOL_NAME'].nunique() > 1:
        duplicates.append(student)
        
    else:
        others.append(student)

non_dup = duplicated_past[duplicated_past['STUDENT_ID'].isin(non_duplicates)]
dup = duplicated_past[duplicated_past['STUDENT_ID'].isin(duplicates)]
other = duplicated_past[duplicated_past['STUDENT_ID'].isin(others)]

In [20]:
df['DISTRICT_CODE'] = df['DISTRICT_CODE'].astype(str).str.zfill(3)
df['LOCATION_ID'] = df['LOCATION_ID'].astype(str).str.zfill(3)

df = pd.concat([attendance_dfs[2019], attendance_dfs[2020], attendance_dfs[2021], attendance_dfs[2022], attendance_dfs[2023], attendance_dfs[2024]], axis=0).reset_index(drop=True)
df.to_csv(config.INTERIM_DATA_DIR / 'Merged_Data.csv', index=False)